# Using Convolutional Neural Networks

In [1]:
%matplotlib inline

In [2]:
# path = "data/dogscats/"
path = "/home/ubuntu/data/dogscats/sample/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
import utils; reload(utils)
from utils import plots

Using Theano backend.


In [5]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [6]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batch_size = 16





In [7]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

vgg.finetune(batches)

vgg.fit(batches, val_batches, nb_epoch=1)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/1
160/160 [==============================] - 75s - loss: 0.8433 - acc: 0.7625 - val_loss: 0.0906 - val_acc: 0.9750


In [8]:
test_batches = vgg.get_batches(path+'valid', shuffle=False, batch_size=2, class_mode=None)


Found 40 images belonging to 2 classes.


In [12]:
test_batches.filenames

['cats/cat.9543.jpg',
 'cats/cat.11290.jpg',
 'cats/cat.77.jpg',
 'cats/cat.6215.jpg',
 'cats/cat.942.jpg',
 'cats/cat.5086.jpg',
 'cats/cat.486.jpg',
 'cats/cat.7710.jpg',
 'cats/cat.2323.jpg',
 'cats/cat.5540.jpg',
 'cats/cat.11750.jpg',
 'cats/cat.3374.jpg',
 'cats/cat.10296.jpg',
 'cats/cat.8051.jpg',
 'cats/cat.6015.jpg',
 'cats/cat.11420.jpg',
 'cats/cat.6261.jpg',
 'cats/cat.9909.jpg',
 'cats/cat.11392.jpg',
 'cats/cat.7929.jpg',
 'dogs/dog.3056.jpg',
 'dogs/dog.4873.jpg',
 'dogs/dog.9144.jpg',
 'dogs/dog.6822.jpg',
 'dogs/dog.11359.jpg',
 'dogs/dog.5338.jpg',
 'dogs/dog.6253.jpg',
 'dogs/dog.4087.jpg',
 'dogs/dog.2702.jpg',
 'dogs/dog.8511.jpg',
 'dogs/dog.9847.jpg',
 'dogs/dog.3719.jpg',
 'dogs/dog.6186.jpg',
 'dogs/dog.11153.jpg',
 'dogs/dog.2056.jpg',
 'dogs/dog.12139.jpg',
 'dogs/dog.10097.jpg',
 'dogs/dog.11518.jpg',
 'dogs/dog.7368.jpg',
 'dogs/dog.11179.jpg']

In [15]:

pred = vgg.model.predict_generator(test_batches, test_batches.nb_sample)

In [23]:
print (type(pred))
print(pred[:,1])

<type 'numpy.ndarray'>
[  1.2764e-03   7.0611e-05   1.8702e-03   7.0584e-02   1.2168e-02   1.0498e-03   1.5010e-02
   2.0017e-03   2.7424e-03   8.9717e-05   1.2753e-04   5.2025e-03   4.6089e-03   1.0343e-05
   1.3803e-04   9.3737e-01   1.4976e-02   7.1235e-04   1.2586e-03   3.9165e-02   9.8250e-01
   9.9985e-01   9.9999e-01   6.1557e-01   9.9054e-01   9.9727e-01   9.3641e-01   1.0000e+00
   9.9895e-01   9.9860e-01   9.9999e-01   9.8911e-01   9.9937e-01   9.9974e-01   9.9977e-01
   9.2948e-01   9.9893e-01   9.9996e-01   9.9934e-01   9.9451e-01]


In [18]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

The category indexes are based on the ordering of categories used in the VGG model - e.g here are the first four:

When the VGG model was trained in 2014, the creators subtracted the average of each of the three (R,G,B) channels first, so that the data for each channel had a mean of zero. Furthermore, their software that expected the channels to be in B,G,R order, whereas Python by default uses R,G,B. We need to preprocess our data to make these two changes, so that it is compatible with the VGG model:

In [28]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

Now we're ready to define the VGG model architecture - look at how simple it is, now that we have the basic blocks defined!

In [35]:
def pred_batch(imgs):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[0, :5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))

In [36]:
pred_batch(imgs)

Shape: (4, 1000)
First 5 classes: [u'tench', u'goldfish', u'great_white_shark', u'tiger_shark', u'hammerhead']
First 5 probabilities: [  7.0631e-07   1.3811e-05   1.0875e-05   2.5794e-06   1.8271e-06]

Predictions prob/class: 
  0.2156/Persian_cat
  0.6678/tiger_cat
  0.5281/miniature_poodle
  0.5970/Labrador_retriever
